## ISO 18431-4

Shock Response Spectra 계산 Coefficient

$y_n = b_0 \cdot x_n + b_1 \cdot x_{n-1} + b_2 \cdot x_{n-2} - a_1 \cdot y_{n-1} - a_2 \cdot y_{n-2}$

$b_0 = 1- exp(-A)\cdot sin(B)/B$

$b_1 = 2exp(-A)\cdot {sin(B)/B - cos(B)}$

$b_2 = exp(-2A)-exp(-A)\cdot sin(B)/B$

$a_1 = -2exp(-A)\cdot cos(B)$

$a_2 = exp(-2A)$

$A = \frac{\omega_n \cdot T}{2Q}$

$B = \omega_n \cdot T \cdot \sqrt{1-\frac{1}{4Q^2}}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter

In [ ]:
wd = wn*np.sqrt(1-zeta**2);
A=wn*T*zeta;
B=wd*T;

b[0] = 1 - np.exp(-A)*np.sin(B)/Bl;
b[1] = 2*np.exp(-A)*(sin(B)/B - cos(B));
b[2] = np.exp(-2A) - np. exp(-A) * np.sin(B)/B;

a[0] = 1;
a[1] = -2*np.exp(-A)*cos(B);
a[2] = np.exp(-2*A);